In [21]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.optimizers import Adam
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


In [22]:
stories = [
""""The Forgotten Map": In a dusty old library, a young researcher discovers a mysterious map hidden within the pages of an ancient book. The map seems to lead to a long-lost kingdom filled with treasures and secrets. As the researcher delves deeper into deciphering the map's clues, they uncover unexpected challenges and adversaries determined to reach the treasure first.""",

""""Whispers in the Woods": In a secluded forest, strange whispers are heard by anyone who dares to enter. A curious teenager, known for their bravery, decides to investigate. As they venture deeper into the woods, they encounter enigmatic creatures and learn that the whispers hold the key to a forgotten prophecy that could change the fate of their world.""",

""""The Last Beacon": In a world plunged into darkness after an ancient catastrophe, a young engineer discovers an ancient lighthouse said to hold the power to restore light to the world. With the help of a group of unlikely allies, the engineer embarks on a perilous journey across treacherous lands, facing formidable challenges and adversaries who seek to keep the world in eternal darkness.""",

""""Echoes of Time": A gifted physicist invents a time-traveling device capable of sending messages to the past. However, the messages sent cause unforeseen ripples through time, altering events in unexpected ways. As the fabric of reality begins to unravel, the physicist races against time to correct the changes before irreparable damage occurs.""",

""""The Dream Catcher": In a town plagued by a series of haunting nightmares, a young artist discovers they have the ability to enter dreams and alter their course. Tasked with unraveling the mystery behind these nightmares, they navigate a surreal dream world, facing manifestations of people's deepest fears and secrets, all while trying to prevent a looming catastrophe that threatens to merge the dream realm with reality.""",
    
    # Add more stories here
]

In [23]:
# this checks the longest story with most number of words
max_len = max(len(story.split()) for story in stories)



67

In [24]:
vocab_size = 10000  # Choose an appropriate vocabulary size
embedding_dim = 100  # Embedding dimension

In [25]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(stories)


In [26]:
sequences = tokenizer.texts_to_sequences(stories)
sequences

[[1,
  22,
  11,
  5,
  2,
  42,
  43,
  44,
  2,
  12,
  23,
  13,
  2,
  45,
  11,
  46,
  47,
  1,
  48,
  4,
  14,
  15,
  49,
  1,
  11,
  50,
  3,
  51,
  3,
  2,
  52,
  53,
  54,
  55,
  9,
  56,
  6,
  24,
  16,
  1,
  23,
  57,
  25,
  17,
  58,
  1,
  59,
  60,
  7,
  61,
  26,
  27,
  6,
  28,
  62,
  3,
  63,
  1,
  64,
  65],
 [18,
  5,
  1,
  29,
  5,
  2,
  66,
  67,
  68,
  18,
  69,
  70,
  30,
  71,
  31,
  72,
  3,
  32,
  2,
  73,
  74,
  75,
  76,
  19,
  77,
  78,
  3,
  79,
  16,
  7,
  80,
  25,
  17,
  1,
  29,
  7,
  81,
  82,
  83,
  6,
  84,
  20,
  1,
  18,
  33,
  1,
  85,
  3,
  2,
  22,
  86,
  20,
  87,
  88,
  1,
  89,
  4,
  19,
  8],
 [1,
  90,
  91,
  5,
  2,
  8,
  92,
  17,
  34,
  93,
  14,
  15,
  35,
  2,
  12,
  36,
  13,
  14,
  15,
  94,
  95,
  3,
  33,
  1,
  96,
  3,
  97,
  98,
  3,
  1,
  8,
  9,
  1,
  99,
  4,
  2,
  100,
  4,
  101,
  102,
  1,
  36,
  103,
  104,
  2,
  105,
  106,
  107,
  108,
  109,
  37,
  110,
  27,
  6,
  28,

In [27]:
# [
#     [1, 2, 3, 4, 1, 5],
#     [1, 6, 7, 8, 9],
#     [10, 11, 12, 4, 1, 13]
# ]
## converts this code to this using
# padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
# [
#     [1, 2, 3, 4, 1, 5],    # Unchanged - already at max_len
#     [1, 6, 7, 8, 9, 0],   # Padded with zeros at the end
#     [10, 11, 12, 4, 1, 13]  # Unchanged - already at max_len
# ]
# # 
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [28]:
# Building the generator model
generator = Sequential()
generator.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
generator.add(Bidirectional(LSTM(128)))
# activation='softmax' is used because the generator's output should represent 
# probabilities for each element in vocab_size in the vocabulary.
generator.add(Dense(vocab_size, activation='softmax'))

# Building the discriminator model
discriminator = Sequential()
discriminator.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
discriminator.add(Bidirectional(LSTM(128)))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Combining generator and discriminator as part of GAN
discriminator.trainable = False
gan_input = generator.input
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [44]:
# Training the GAN
epochs = 100 # Choose an appropriate number of epochs
batch_size = 32  # Define batch size
for epoch in range(epochs):
    noise = np.random.randint(0, vocab_size, size=(batch_size, max_len))
    generated_stories = generator.predict(noise)
    real_stories = padded_sequences[np.random.randint(0, len(padded_sequences), size=batch_size)]
    print(epoch)
    
    





1/1 [==============================] - 0s 53ms/step
0
1/1 [==============================] - 0s 59ms/step
1
1/1 [==============================] - 0s 56ms/step
2
1/1 [==============================] - 0s 51ms/step
3
1/1 [==============================] - 0s 53ms/step
4
1/1 [==============================] - 0s 62ms/step
5
1/1 [==============================] - 0s 56ms/step
6
1/1 [==============================] - 0s 55ms/step
7
1/1 [==============================] - 0s 65ms/step
8
1/1 [==============================] - 0s 92ms/step
9
1/1 [==============================] - 0s 76ms/step
10
1/1 [==============================] - 0s 80ms/step
11
1/1 [==============================] - 0s 74ms/step
12
1/1 [==============================] - 0s 78ms/step
13
1/1 [==============================] - 0s 74ms/step
14
1/1 [==============================] - 0s 61ms/step
15
1/1 [==============================] - 0s 53ms/step
16
1/1 [==============================] - 0s 59ms/step
17
1/1 [===============

In [46]:
real_stories.shape

(32, 67)

1/1 [==============================] - 1s 935ms/step


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 67 and the array at index 1 has size 10000